#  Consulting Project 

You've been hired by a dog food company to try to predict why some batches of their dog food are spoiling much quicker than intended! Unfortunately this Dog Food company hasn't upgraded to the latest machinery, meaning that the amounts of the five preservative chemicals they are using can vary a lot, but which is the chemical that has the strongest effect? The dog food company first mixes up a batch of preservative that contains 4 different preservative chemicals (A,B,C,D) and then is completed with a "filler" chemical. The food scientists beelive one of the A,B,C, or D preservatives is causing the problem, but need your help to figure out which one!
Use Machine Learning with RF to find out which parameter had the most predicitive power, thus finding out which chemical causes the early spoiling! So create a model and then find out how you can decide which chemical is the problem!

* Pres_A : Percentage of preservative A in the mix
* Pres_B : Percentage of preservative B in the mix
* Pres_C : Percentage of preservative C in the mix
* Pres_D : Percentage of preservative D in the mix
* Spoiled: Label indicating whether or not the dog food batch was spoiled.
___

In [1]:
#import the necessary libraries
from pyspark.sql import SparkSession
import pandas as pd
from pyspark.ml.feature import StandardScaler

#start a spark session
spark = SparkSession.builder.appName('dogfoodpoison').getOrCreate()

## Exploratory Analysis

In [2]:
#record the findings
findings = []

In [3]:
#load the dataset
data = spark.read.csv('13.dog_food.csv', inferSchema=True, header=True)
pd.DataFrame(data.take(3), columns=data.columns)

,A,B,C,D,Spoiled
0,4,2,12.0,3,1.0
1,5,6,12.0,7,1.0
2,6,2,13.0,6,1.0


In [4]:
#analyze the basic stats of the columns
pd.DataFrame(data.summary().collect(), columns=data.summary().columns)

,summary,A,B,C,D,Spoiled
0,count,490,490,490,490,490
1,mean,5.53469387755102,5.504081632653061,9.126530612244897,5.579591836734694,0.2857142857142857
2,stddev,2.9515204234399057,2.8537966089662063,2.0555451971054275,2.8548369309982857,0.45221563164613465
3,min,1,1,5.0,1,0.0
4,25%,3,3,8.0,3,0.0
5,50%,5,6,9.0,6,0.0
6,75%,8,8,11.0,8,1.0
7,max,10,10,14.0,10,1.0


In [5]:
#record the findings
findings += ['#Chemical C values have a differnet range than the rest', 
             '#Spoiled is the target value',
             '#No missing values',
             '#Features should be standardized']

In [6]:
data.printSchema()

root
 |-- A: integer (nullable = true)
 |-- B: integer (nullable = true)
 |-- C: double (nullable = true)
 |-- D: integer (nullable = true)
 |-- Spoiled: double (nullable = true)



In [7]:
data.columns

['A', 'B', 'C', 'D', 'Spoiled']

# Preprocess the data

In [8]:
#initiate workflow pipeline
stages = []

### Assembling the features into a vector

In [9]:
#prepare the input vectors
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import (VectorAssembler, VectorIndexer)
assembler = VectorAssembler(inputCols=['A', 'B', 'C', 'D'], outputCol='feature_unscaled')

In [10]:
data = assembler.transform(data)

### Scaling the features

In [11]:
#prepare the scaler
from pyspark.ml.feature import StandardScaler
scaler = StandardScaler(inputCol='feature_unscaled', outputCol='features')

In [12]:
data = scaler.fit(data).transform(data)

### Split the data into Train and Test Data

In [13]:
#splitting the 
train_data, test_data = data.randomSplit([0.7, 0.3])

In [14]:
train_data.columns

['A', 'B', 'C', 'D', 'Spoiled', 'feature_unscaled', 'features']

In [15]:
train_data.head(1)

[Row(A=1, B=1, C=10.0, D=8, Spoiled=1.0, feature_unscaled=DenseVector([1.0, 1.0, 10.0, 8.0]), features=DenseVector([0.3388, 0.3504, 4.8649, 2.8023]))]

In [16]:
train_data.summary('count').show()

+-------+---+---+---+---+-------+
|summary|  A|  B|  C|  D|Spoiled|
+-------+---+---+---+---+-------+
|  count|356|356|356|356|    356|
+-------+---+---+---+---+-------+



In [17]:
test_data.columns

['A', 'B', 'C', 'D', 'Spoiled', 'feature_unscaled', 'features']

In [18]:
test_data.summary('count').show()

+-------+---+---+---+---+-------+
|summary|  A|  B|  C|  D|Spoiled|
+-------+---+---+---+---+-------+
|  count|134|134|134|134|    134|
+-------+---+---+---+---+-------+



In [19]:
test_data.printSchema()

root
 |-- A: integer (nullable = true)
 |-- B: integer (nullable = true)
 |-- C: double (nullable = true)
 |-- D: integer (nullable = true)
 |-- Spoiled: double (nullable = true)
 |-- feature_unscaled: vector (nullable = true)
 |-- features: vector (nullable = true)



In [20]:
train_data.printSchema()

root
 |-- A: integer (nullable = true)
 |-- B: integer (nullable = true)
 |-- C: double (nullable = true)
 |-- D: integer (nullable = true)
 |-- Spoiled: double (nullable = true)
 |-- feature_unscaled: vector (nullable = true)
 |-- features: vector (nullable = true)



## Build the ML Classification Models

In [21]:

#prepare the classification models
from pyspark.ml.classification import (RandomForestClassifier, GBTClassifier, DecisionTreeClassifier)

#random forest classification
rfc = RandomForestClassifier(labelCol='Spoiled', predictionCol='rfc_prediction',
                             rawPredictionCol='rawPrediction_rfc', probabilityCol='probability_rfc')
stages += [rfc]

#gradient boosted tree classification
gbtc = GBTClassifier(labelCol='Spoiled', predictionCol='gbtc_prediction')
stages += [gbtc]

#decison tree classification
dtc = DecisionTreeClassifier(labelCol='Spoiled', predictionCol='dtc_prediction',
                             rawPredictionCol='rawPrediction_dtc', probabilityCol='probability_dtc')
stages += [dtc]


## Train the model

In [22]:
stages

[RandomForestClassifier_631ae53bd7ba,
 GBTClassifier_883677b56057,
 DecisionTreeClassifier_d8d91580a72c]

In [23]:
#prepare the pipeline
from pyspark.ml import Pipeline
#prepare the pipeline
pipeline = Pipeline(stages=stages)

In [24]:
#Training the Data
models = pipeline.fit(train_data)

In [25]:
results = models.transform(test_data)

## Evaluate the Models

In [26]:
#importing the evaluators
from pyspark.ml.evaluation import (BinaryClassificationEvaluator, MulticlassClassificationEvaluator)

In [27]:
#Evaluation of Random Forest models
evaluate_rfc = MulticlassClassificationEvaluator(labelCol='Spoiled', 
                                                 predictionCol='rfc_prediction',
                                                 metricName='accuracy')

#RFC Accuracy
evaluate_rfc.evaluate(results)

0.9776119402985075

In [28]:
#Evaluation of GBTC Model
evaluate_gbtc = MulticlassClassificationEvaluator(labelCol='Spoiled', 
                                                 predictionCol='gbtc_prediction',
                                                 metricName='accuracy')

#RFC Accuracy
evaluate_gbtc.evaluate(results)

0.9552238805970149

In [29]:
#Evaluation of Random Forest models
evaluate_dtc = MulticlassClassificationEvaluator(labelCol='Spoiled', 
                                                 predictionCol='dtc_prediction',
                                                 metricName='accuracy')

#RFC Accuracy
evaluate_dtc.evaluate(results)

0.9552238805970149

In [30]:
pd.DataFrame(results.take(2), columns=results.columns).T

,0,1
A,1,1
B,1,2
C,12,9
D,4,1
Spoiled,1,0
feature_unscaled,"[1.0, 1.0, 12.0, 4.0]","[1.0, 2.0, 9.0, 1.0]"
features,"[0.3388084297362005, 0.35041039605210417, 5.83...","[0.3388084297362005, 0.7008207921042083, 4.378..."
rawPrediction_rfc,"[0.0, 20.0]","[19.2640350877193, 0.7359649122807017]"
probability_rfc,"[0.0, 1.0]","[0.963201754385965, 0.03679824561403509]"
rfc_prediction,1,0


## Determine which chemical causes the food to spoil
### The chemical with the most weight is the one we are looking for

In [31]:
rfc_model = RandomForestClassifier(labelCol='Spoiled').fit(train_data )
rfc_feature_weights = rfc_model.featureImportances
rfc_feature_weights

SparseVector(4, {0: 0.034, 1: 0.0284, 2: 0.9039, 3: 0.0338})

### Result doesn't change with other classification model
### The chemical at index 2 is Chemical C, and it has the greatest impact/weight. Therefore it is the one causing food to spoil.